# Documentation of the python script for the text encoding in TEI XML

In the context of the DAHN project, we are developing a pipeline for the digital edition of textual documents. There are multiples steps to this pipeline (digitization, segmentation, transcription, OCR post-processing, encoding, publication) and the goal is to computerize as much as possible the steps, so that the researcher have less work to do and more time to do more.  
Here, I am presenting the fifth step of our pipeline : the TEI XML encoding. The last step of our pipeline uses the TEI Publisher tool, which requires the import of TEI XML files to work. In any case, having a corpus in a machine readable format is always useful and important. However, encoding a corpus can be a long and onerous task, even more if the corpus is large and dense. To overcome this problem, I created a Python script, which goal is to ease and fasten the encoding by using the structure of the TEXT format of the corpus and some redundant parts of the text (which are massively present in our study text : egodocuments).  
By using that script, a huge part of the encoding is already made for all the files in the corpus just by writing one line in the command line interface. Then, a correction and supervision will be mandatory but it will take much less time that it would have take to encode all the texts manually.  
Some parts of the script presented here are specific to the text used as an example but I will also show you how to use and customize it for your own corpus. My example text is a letter from the corpus used as a case study to develop the whole pipeline, as well as the following script.

## Composition of the script

### Importation of the modules

Firstly, we have to import some modules to execute the script.

In [1]:
import os
import re
import sys 

We import three modules : `os` and `sys` are used for calling files in the script and in the command line (__They will not be used in the executable part of the jupyter notebook script__).  
The module `re` is the most important import in the script since it will be responsible for encoding key elements in the text, by using regular expressions. 

### Encoding linebreaks by using a constant

When encoding, the text is usually not contiguous for the start to the end of the paragraph and the edition provides some linebreaks to ease the reading of the text. Those linebreak are usually marked by a newline in the TEXT format, which will eased the encoding, because we simply transform the newline (\n) in linebreak tags (`<lb/>`).  
Thoses transformations are constant and does not need changes (except those you will do when you customize your own script) so we put them in a constant, which require to be declared in capital letters.  
There are two types of linebreaks : one where it is simply a newline after a space (or some other characters like a comma) and one where a word has been cut in two. In this case, it is important to add an attribute to the tag (`<lb break="no"/>`) to specify this particular case. From there, you can choose to keep or remove the separation sign before the tag.

In [2]:
LINEBREAK = {
    '-\n': '-<lb break="no"/>',
    ' \n': '<lb/> ',
    "'\n": "'<lb/>"
}

### Encoding paragraphs through a function

This first function is the most generic one and it should work for every type of text (but it can be changed with more adequate tags if necessary, like a `<lg>`for a poem).  
It considers the structure given to the TEXT format it reads and add the tags for paragraphs when it meets the prerequisites given in the script. In this function, the variable `punctuation` has been filled with the type of punctuation found in the study corpus, but it can adapted to your own corpus.

In [3]:
def tagging_paragraph(text):
    """ Add paragraph tags in a text
    
    :param text str: text that has to be modified
    :returns: text encoded with paragraph tags
    :rtype: str
    """
    
    punctuation = "!\"»?." 
    for sign in punctuation:
        text = text.replace(sign + "\n", sign + '</p>\n<p rend="indent">')
        text = text.replace(sign + " \n", sign + '</p>\n<p rend="indent">')
    return text

This function, however, generates some errors in the encoding that will necessite some manual corrections afterwards, because the way it works is by thinking that when a paragraph ends, another starts right after and it is not always the case. So, during the correction phase, it will most likely be necessary to suppress some new paragraph tags created.

### Encoding redundants elements of the text with a function and regular expressions

This function relies mainly on regular expressions to work and it needs to be customized in order to be effective for your corpus. It is composed of two series of statements : the first one declares the expressions that need to be found in the read text and the second one encodes what it will have found in the text.

To help you with the creation of your regular expression (or regex), here are some useful characters :  
- ? : the character indicates zero or one occurences
- \* : the character indicates zero or more occurences
- \+ : the character indicates one or more occurences
- ^ : start of a lign
- $ : end onf a lign
- | : act as a 'or' in the declaration
- () : defines the scope of the operation
- {} : specify how many times a character appears
- [] : contains the characters inside the brackets
- [A-Za-zÀ-ÖØ-öø-ÿ-] : letters from the alphabet with accents (for non-english corpus)
- [0-9] : contains all the number from 0 to 9  

To have more informations on regex, you can also see [regexr.com](https://regexr.com).

For example, in our corpus (a correspondence), the author of the letter always finishes with a polite phrase. It is usually the same phrase but sometimes, the wording slightly changes. In this case, we can use a regex to take thoses changes into consideration and encode the phrase, even though it differs.  
The statement will be as such : `signature = re.compile((r'^(A|a)ffectueusement (à vous|votre),?')` and this results in :  
- Affectueusement à vous,
- affectueusement à vous,
- Affectueusement à vous
- affectueusement à vous
- Affectueusement votre,
- affectueusement votre,
- Affectueusement votre
- affectueusement votre  

The changes are slim but they all are going to be considered during the encoding.

Two other important informations for the redaction and encoding of regex :
- It is important to use the python raw string `r' '` or the regular expression will not be properly read.
- In the second series of statements, while encoding, it is necessary to put between the tags, and as representation of the regex that is called, the following expression: `\g<0>`. This expression represent the interpretation of the regex found in the text.  

Subsequently, to encode this signature, we have to declare the following thing in the second series of statement :  
`text = re.sub(signature, r'<closer><signed rend="align(right)">\g<0></signed>', text)`  
It will encode what it will find in the text, as long as it corresponds to one of the eight propositions above.

In [4]:
def tagging_regex(text):
    """Apply tags to the regex

    :param text str: text that has to be modified
    :returns: text encoded with the right tags associated to a regex
    :rtype: str
    """
    
    #name_of_the_regex = re.compile(regex)
    letter = re.compile(r'^(Annexe à ma )?L(ETTRE N|ettre n)° ?[0-9]+ ?.?')
    senate = re.compile(r'^S((E|É)NAT|énat)')
    dateline = re.compile(r'^[A-Za-zÀ-ÖØ-öø-ÿ-]+(( |-)[A-Za-zÀ-ÖØ-öø-ÿ-]+)?, (le )?[0-9]* [A-Za-zÀ-ÖØ-öø-ÿ-]+ 19[1-2][0-9] ?.?')
    salute = re.compile(r'^Mon cher Butler ?,')
    adress = re.compile(r'^(à )?Monsieur le Président N(.)?(icholas)? ?Murray BUTLER.?')
    addrline = re.compile(r'^(NEW) ?(-| )? ?(YORK).?$')
    signature = re.compile(r'^Votre [A-Za-zÀ-ÖØ-öø-ÿ-]+ dévoué.?')
    name = re.compile(r"(D|d)'E(stournelles|STOURNELLES)( de C(onstant|ONSTANT))? ?.?")
    annexe = re.compile(r'^[0-9]* ?(A|a)nnexe(s)?.?')
    deletion = re.compile(r'€[A-Za-zÀ-ÖØ-öø-ÿ-]*€')
    
    #text = re.sub(name_of_the_regex, r'<new_tag>\g<0></new_tag>', text)
    text = re.sub(letter, r'<head rend="center underline">\g<0></head><opener>', text)
    text = re.sub(senate, r'<fw type="letterhead" place="margin" corresp="#lh-senat"><hi rend="underline">\g<0></hi></fw>', text)
    text = re.sub(dateline, r'<dateline rend="align(right)">\g<0></dateline>', text)
    text = re.sub(salute, r'<salute rend="indent">\g<0></salute></opener><p rend="indent">', text)
    text = re.sub(adress, r'<address><addrLine rend="margin">\g<0></addrLine>', text)
    text = re.sub(addrline, r'<addrLine rend="indent"><hi rend="underline">\g<0></hi></addrLine></address>', text)
    text = re.sub(signature, r'<closer><signed rend="align(right)">\g<0></signed>', text)
    text = re.sub(name, r'<signed rend="align(right)" hand="#annotation">\g<0></signed>', text)
    text = re.sub(annexe, r'<postscript><p rend="bottom">\g<0></p></postscript>', text)
    text = re.sub(deletion, r'<del rend="strikethrough">\g<0></del>', text)
    return text

All the regex declared above correspond to elements that are going to be found in the example text. It is recommanded to use this kind of function if you have several redundant elements in your own corpus. Otherwise, it would be useless and you can erase this part from your script. 

### Calling the TEXT format of the files

After declaring our functions and dictionnaries, we can start with the transformation script, where we are going to apply what we just defined.  
For the purpose of the notebook, the script will be used with only one text but the official script (available [here](https://github.com/FloChiff/DAHNProject/blob/master/Project%20development/Scripts/Encoding/text_tagging.py)) is usable with folder of multiples files. This script will also be provided in the notebook but will not be executable.

In [5]:
#for root, dirs, files in os.walk(sys.argv[1]):
#    for filename in files:
#        with open(sys.argv[1] + filename, 'r') as file_in:
#            print("reading from "+sys.argv[1] + filename)
#            file = file_in.read()
#        file = file.replace("\n", "\n$"`

The first step is to call our folder and files. The TEXT format files are read and are treated as a whole. In it, the newline are kept and signify by the character `\n`. To help with one of the following steps, I added a new character next to the newline, a $.

In [6]:
input_var = "LETTRE N° 601.\nSENAT\nPARIS, 15 Décembre 1919.\nSOMMAIRE DE MES LETTRES\ndu N° 251 au N° 600 inclus.\nMon cher Butler,\nJe vous envoie ci-joint le sommaire des lettres\nque je vous ai adressées au cours du €premier€ second semestre de 1919,\ndu N° 551 au N° 600 inclus.\nVotre affectueusement dévoué,\n£d'Estournelles£\nà Monsieur le Président Nicholas Murray BUTLER,\nNEW-YORK.\nannexe."

In [7]:
file = input_var.replace("\n", "\n$")
print(file)

LETTRE N° 601.
$SENAT
$PARIS, 15 Décembre 1919.
$SOMMAIRE DE MES LETTRES
$du N° 251 au N° 600 inclus.
$Mon cher Butler,
$Je vous envoie ci-joint le sommaire des lettres
$que je vous ai adressées au cours du €premier€ second semestre de 1919,
$du N° 551 au N° 600 inclus.
$Votre affectueusement dévoué,
$£d'Estournelles£
$à Monsieur le Président Nicholas Murray BUTLER,
$NEW-YORK.
$annexe.


### Transformation of the files

#### Splitting the text into a list

To transform the text, we read it line by line, as it makes its transformation easier. Consequently, we start by creating a empty list in which our transformed files will be added at the end. We also create a list in which we find our text, where a line corresponds to an entry in the list. To do so, we used the .split() method.  

The python method split() split a string into a list and the character between parenthesis specify the separator in the string. In our case, since we need to keep the newline to encode our text, I previously add a random character (__Attention: this character should not be present in your corpus or it will distort the processus__) and I split the whole text with this character as the separator. 

In [8]:
processed_text_as_list = []
text_as_list = file.split('$')
print(text_as_list)

['LETTRE N° 601.\n', 'SENAT\n', 'PARIS, 15 Décembre 1919.\n', 'SOMMAIRE DE MES LETTRES\n', 'du N° 251 au N° 600 inclus.\n', 'Mon cher Butler,\n', 'Je vous envoie ci-joint le sommaire des lettres\n', 'que je vous ai adressées au cours du €premier€ second semestre de 1919,\n', 'du N° 551 au N° 600 inclus.\n', 'Votre affectueusement dévoué,\n', "£d'Estournelles£\n", 'à Monsieur le Président Nicholas Murray BUTLER,\n', 'NEW-YORK.\n', 'annexe.']


#### Executing the defined functions

In the context of the notebook, I create a new list everytime I present a block of the script, to help with the understanding and to still have a functionning loop, but it will not be necessary when writing the script as a whole, as you will see at the end of the demonstration.

The first step in the loop is not necessary but can be useful to homogenize your punctuation, especially if you use a specific apostrophe in your function (if it is not the same, it will not consider it).  
Then, we apply the two functions that we created. It is not important when writing the script if you declare first the paragraph function or the regex function ; however, when applying it to your corpus, it is very important to execute the regex function first, then the paragraph function. Otherwise, some regex can't be consider and most of the encoding will be deformed.

In [9]:
list1 = []
for text in text_as_list:
    text = text.replace("’", "'")
    text = tagging_regex(text)
    text = tagging_paragraph(text)
    list1.append(text)
    print(text)

<head rend="center underline">LETTRE N° 601.</head><opener>

<fw type="letterhead" place="margin" corresp="#lh-senat"><hi rend="underline">SENAT</hi></fw>

<dateline rend="align(right)">PARIS, 15 Décembre 1919.</dateline>

SOMMAIRE DE MES LETTRES

du N° 251 au N° 600 inclus.</p>
<p rend="indent">
<salute rend="indent">Mon cher Butler,</salute></opener><p rend="indent">

Je vous envoie ci-joint le sommaire des lettres

que je vous ai adressées au cours du <del rend="strikethrough">€premier€</del> second semestre de 1919,

du N° 551 au N° 600 inclus.</p>
<p rend="indent">
<closer><signed rend="align(right)">Votre affectueusement dévoué,</signed>

£<signed rend="align(right)" hand="#annotation">d'Estournelles£</signed>

<address><addrLine rend="margin">à Monsieur le Président Nicholas Murray BUTLER,</addrLine>

<addrLine rend="indent"><hi rend="underline">NEW-YORK.</hi></addrLine></address>

<postscript><p rend="bottom">annexe.</p></postscript>


#### Applying the linebreak changes

In this part, we are encoding the linebreaks of the text. Firstly, we are using the dictionary previously defined.  
Then, we are searching in a very special case to encode the rest of the linebreak. As the text is read as single lines in a loop, the goal is to encode linebreak in the lines where no encoding has already been tagged, mostly with the regex, encoded earlier in the script. So, if there is no markup in the line but still a newline `\n`, a linebreak tag will be added.

In [10]:
list2 = []
for text in list1:
    for key, value in LINEBREAK.items():
        text = text.replace(key, value)
    if ">" not in text:
        text = text.replace("\n","<lb/> ")
    list2.append(text)
    print(text)

<head rend="center underline">LETTRE N° 601.</head><opener>

<fw type="letterhead" place="margin" corresp="#lh-senat"><hi rend="underline">SENAT</hi></fw>

<dateline rend="align(right)">PARIS, 15 Décembre 1919.</dateline>

SOMMAIRE DE MES LETTRES<lb/> 
du N° 251 au N° 600 inclus.</p>
<p rend="indent">
<salute rend="indent">Mon cher Butler,</salute></opener><p rend="indent">

Je vous envoie ci-joint le sommaire des lettres<lb/> 
que je vous ai adressées au cours du <del rend="strikethrough">€premier€</del> second semestre de 1919,

du N° 551 au N° 600 inclus.</p>
<p rend="indent">
<closer><signed rend="align(right)">Votre affectueusement dévoué,</signed>

£<signed rend="align(right)" hand="#annotation">d'Estournelles£</signed>

<address><addrLine rend="margin">à Monsieur le Président Nicholas Murray BUTLER,</addrLine>

<addrLine rend="indent"><hi rend="underline">NEW-YORK.</hi></addrLine></address>

<postscript><p rend="bottom">annexe.</p></postscript>


#### Final cleaning of the text

Finally, this last part is for specific part of the transformed text, used in the transcription but not inherent to the real text. It can be changed according to the way your text have been encoded. In our case, during our transcription and to help with the encoding (mostly with the regex), we found a way to point out deletions (`€`) and additions (`£`), by adding unique characters included in the regex. Once the encoding has been done, those characters are no longer useful so they have to be removed from the text.  
We then add the transformed lines of our text in the list that we created at the beginning of the script with the append() method.

In [11]:
for text in list2:
    text = text.replace("£", "")
    text = text.replace("€", "")
    processed_text_as_list.append(text)
    print(text)

<head rend="center underline">LETTRE N° 601.</head><opener>

<fw type="letterhead" place="margin" corresp="#lh-senat"><hi rend="underline">SENAT</hi></fw>

<dateline rend="align(right)">PARIS, 15 Décembre 1919.</dateline>

SOMMAIRE DE MES LETTRES<lb/> 
du N° 251 au N° 600 inclus.</p>
<p rend="indent">
<salute rend="indent">Mon cher Butler,</salute></opener><p rend="indent">

Je vous envoie ci-joint le sommaire des lettres<lb/> 
que je vous ai adressées au cours du <del rend="strikethrough">premier</del> second semestre de 1919,

du N° 551 au N° 600 inclus.</p>
<p rend="indent">
<closer><signed rend="align(right)">Votre affectueusement dévoué,</signed>

<signed rend="align(right)" hand="#annotation">d'Estournelles</signed>

<address><addrLine rend="margin">à Monsieur le Président Nicholas Murray BUTLER,</addrLine>

<addrLine rend="indent"><hi rend="underline">NEW-YORK.</hi></addrLine></address>

<postscript><p rend="bottom">annexe.</p></postscript>


### Creating and writing the files with their TEI XML encoding

After transforming our files, we have to add them to a new file, this time in XML.  
First, it is necessary to transform our list in a string (or we wouldn't be able to write them in a new file) by using the join() method.  
Then, we open (and create) the new file (which path has been put in a variable to avoid the repetition of a long  declaration) and write our transformed text in it.

In [12]:
#new_text = "".join(processed_text_as_list)
#output_file = sys.argv[2] + filename.replace(".txt", ".xml")
#with open(output_file, "w") as file_out:
#    print("writing to " + output_file)
#    output_file.write(new_text)

In [13]:
output_var = ""
output_var = "".join(processed_text_as_list)
print(output_var)

<head rend="center underline">LETTRE N° 601.</head><opener>
<fw type="letterhead" place="margin" corresp="#lh-senat"><hi rend="underline">SENAT</hi></fw>
<dateline rend="align(right)">PARIS, 15 Décembre 1919.</dateline>
SOMMAIRE DE MES LETTRES<lb/> du N° 251 au N° 600 inclus.</p>
<p rend="indent"><salute rend="indent">Mon cher Butler,</salute></opener><p rend="indent">
Je vous envoie ci-joint le sommaire des lettres<lb/> que je vous ai adressées au cours du <del rend="strikethrough">premier</del> second semestre de 1919,
du N° 551 au N° 600 inclus.</p>
<p rend="indent"><closer><signed rend="align(right)">Votre affectueusement dévoué,</signed>
<signed rend="align(right)" hand="#annotation">d'Estournelles</signed>
<address><addrLine rend="margin">à Monsieur le Président Nicholas Murray BUTLER,</addrLine>
<addrLine rend="indent"><hi rend="underline">NEW-YORK.</hi></addrLine></address>
<postscript><p rend="bottom">annexe.</p></postscript>


### The official script

The demonstration was made with only one file, to help with the comprehension of the script execution but it is actually made to be executed with a folder of files, as such :

In [14]:
#for root, dirs, files in os.walk(sys.argv[1]):
#    for filename in files:
#        with open(sys.argv[1] + filename, 'r') as file_in:
#            print("reading from "+sys.argv[1] + filename)
#            file = file_in.read()
#        file = file.replace("\n", "\n$")
#        processed_text_as_list = []
#        text_as_list = file.split('$')
#        for text in text_as_list:
#            text = text.replace("’", "'")
#            text = tagging_regex(text)
#            text = tagging_paragraph(text)
#            for key, value in LINEBREAK.items():
#                text = text.replace(key, value)
#            if ">" not in text:
#                text = text.replace("\n","<lb/> ")  
#            text = text.replace("£", "")
#            text = text.replace("€", "")
#            processed_text_as_list.append(text)
#        new_text = "".join(processed_text_as_list)
#
#        output_file = sys.argv[2] + filename.replace(".txt", ".xml")
#        with open(output_file, "w") as file_out:
#            print("writing to " + output_file)
#            file_out.write(new_text)

## Execution of the script

Once you customize the script to your need, you will need to execute it. To do that, the command line will be pretty simple.

>```python script.py text/ xml/```

The last two arguments from this command line are the one called in the script. With the module `sys`, the numbering start at 0, just like in Python, so by calling `sys.argv[1]` at the start of the script, we are really calling the second argument in the command line, since the first one will actually be the script. So, in this case, the second argument is the folder that contain the files of the corpus in their TEXT format. The third is the folder that will contain the files of the corpus in their TEI XML format, once we created them by executing the script.